# Advent of Code Day 20
#### Problem in full can be found here: https://adventofcode.com/2023/day/20

In [182]:
# Part 1
# Given an input file that contains a broadcaster, conjunctions, and flip flops that all have connections in
# and out where they send low and high pulses to each other. There is a button that initiates the broadcaster
# To send a pulse to all its connections and repeats the cycle. The state of high pulses / low pulses is saved
# so every time the button is pressed, a new result will appear. Find the amount of high pulses * low pulses
# after the button has been pressed 1000 times
from collections import defaultdict, namedtuple

# Gather the input from the input file
file = open("inputday20.txt")
flip_flops = set()
conjunctions = set()
connections_out = defaultdict(list)
connections_in = defaultdict(list)
for line in file:
    line = line.strip()
    # First half is the broadcaster and the second half are the receivers
    broadcaster, receivers = line.split(" -> ")
    connections_out[broadcaster[1:]] = [e.strip() for e in receivers.split(",")]
    for e in [e.strip() for e in receivers.split(",")]:
        connections_in[e].append(broadcaster[1:])
    if broadcaster[0] == "%":
        flip_flops.add(broadcaster[1:])
    elif broadcaster[0] == "&":
        conjunctions.add(broadcaster[1:])
file.close()

# Set the states for the flip flops and conjunctions and set them all to 0 (low)
flip_flop_state = {f: 0 for f in flip_flops}
conjunction_state = {}
for c in conjunctions:
    conjunction_state[c] = {inp: 0 for inp in connections_in[c]}
state = flip_flop_state, conjunction_state

high, low = 0, 0
# namedtuple acts like a class to require Pulse to have certain variables
Pulse = namedtuple("Pulse", ["sender", "recipient", "is_high"])
# Click the button 1000 times
for i in range(1000):
    # Initialize the queue
    q, pulses = [Pulse("button", "roadcaster", 0)], [] # roadcaster because the b gets cut out in the file parsing
    # While the q is active
    while q:
        # Pop from the queue
        p = q.pop(0)
        # Append to pulses
        pulses.append(p)
        # If the pulse is a flip flop
        if p.recipient in flip_flops:
            # If the pulse is low
            if not p.is_high:
                # Set the state of the pulse to the opposite 0 -> 1, 1 -> 0
                state[0][p.recipient] = not state[0][p.recipient]
                # Add all connections from out to the queue
                for out in connections_out[p.recipient]:
                    q.append(Pulse(p.recipient, out, state[0][p.recipient]))
        # If the pulse is a conjunction
        elif p.recipient in conjunctions:
            # Set the state of the pulse to the current state
            state[1][p.recipient][p.sender] = p.is_high
            # The pulse to send will be 0 (low) if all states are 1 (high) and 1 (high) if not all states are 1 (high)
            to_send = not all(state[1][p.recipient].values())
            for out in connections_out[p.recipient]:
                q.append(Pulse(p.recipient, out, to_send))
        # If the pulse is from the broadcaster
        else:
            # The broadcaster pulse is always low (0)
            for out in connections_out[p.recipient]:
                q.append(Pulse(p.recipient, out, 0))
    # Sum up the low and high from the pulses after the cycle
    low += sum(1 for p in pulses if not p.is_high)
    high += sum(1 for p in pulses if p.is_high)
print(high*low)

861743850


In [183]:
# Part 2
# Restart the states and find the lowest amount of button clicks to deliver a singe low pulse to 'rx'
from math import lcm

# Restarting the states
flip_flop_state = {f: 0 for f in flip_flops}
conjunction_state = {}
for c in conjunctions:
    conjunction_state[c] = {inp: 0 for inp in connections_in[c]}
state = flip_flop_state, conjunction_state

# Only one connection to get to 'rx' which is through 'jz' (connections_in["rx"][0]) so to_watch lists
# all the gates to get to jz
to_watch = connections_in[connections_in["rx"][0]]
first_high_pulse = {x: 0 for x in to_watch}
count = 0
# Loop while atleast one variable still has 0 high pulses
while not all(first_high_pulse.values()):
    # One button press
    count += 1
    # Initialize the queue
    q, pulses = [Pulse("button", "roadcaster", 0)], [] # roadcaster because the b gets cut out in the file parsing
    # While the q is active
    while q:
        # Pop from the queue
        p = q.pop(0)
        # Append to pulses
        pulses.append(p)
        # If the pulse is a flip flop
        if p.recipient in flip_flops:
            # If the pulse is low
            if not p.is_high:
                # Set the state of the pulse to the opposite 0 -> 1, 1 -> 0
                state[0][p.recipient] = not state[0][p.recipient]
                # Add all connections from out to the queue
                for out in connections_out[p.recipient]:
                    q.append(Pulse(p.recipient, out, state[0][p.recipient]))
        # If the pulse is a conjunction
        elif p.recipient in conjunctions:
            # Set the state of the pulse to the current state
            state[1][p.recipient][p.sender] = p.is_high
            # The pulse to send will be 0 (low) if all states are 1 (high) and 1 (high) if not all states are 1 (high)
            to_send = not all(state[1][p.recipient].values())
            for out in connections_out[p.recipient]:
                q.append(Pulse(p.recipient, out, to_send))
        # If the pulse is from the broadcaster
        else:
            # The broadcaster pulse is always low (0)
            for out in connections_out[p.recipient]:
                q.append(Pulse(p.recipient, out, 0))

        # After a cycle of button press is completed, check the pulses
        for p in pulses:
            # If the pulse is high, is a connection to rx and hasn't been found already
            if p.is_high and p.sender in to_watch and not first_high_pulse[p.sender]:
                # Set the first high pulse to the current count of button presses
                first_high_pulse[p[0]] = count

# Once the first occurence of high pulses has been found for each connection to 'jz', can find
# the first occurence of a low pulse being sent to 'rx' by finding the LCM (the first occurence where
# all pulses are high at the same time)
print(lcm(*first_high_pulse.values()))

247023644760071
